In [ ]:
import os
import numpy as np

from matplotlib.image import imread
import matplotlib.pyplot as plt


import scipy
from scipy.spatial.distance import cdist

import cv2
import moviepy.editor as mp
from moviepy.editor import VideoFileClip

from sklearn.mixture import GaussianMixture
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics.pairwise import cosine_similarity

from tqdm import tqdm
import csv
import glob

import warnings

warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

resnet18 = models.resnet18(pretrained=True)
modules18 = list(resnet18.children())[:-1]
model = nn.Sequential(*modules18)
flatten = nn.Flatten()

In [ ]:
def extract_features_resnet18(video_path):
    img_features = {}
    video = cv2.VideoCapture(video_path)
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    fps = video.get(cv2.CAP_PROP_FPS)

    print("Frames per second: ", fps)

    count = 0

    with tqdm(total=frame_count, desc="Extracting Features") as pbar:
        while video.isOpened():
            ret, frame = video.read()
            count += 1
            if not ret:
                break
            img = frame
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (250, 250))
            img = torch.Tensor(img)
            img = img.permute(2, 0, 1)
            img = img.unsqueeze(0)
            features = model(img).view(512)
            features = features.detach().numpy()
            img_features[count] = features
            pbar.update(1)
            if count % 500 == 0:
                pbar.set_postfix({"Processed frames": count})

        video.release()
        print("\n")
        return img_features


def eratosthenis(img_features, n):
    prime_numbers = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]
    segments = [[] for _ in range(n)]

    for fn, fv in img_features.items():
        frame_number = fn
        assigned = False
        for i in range(n):
            if frame_number % prime_numbers[i] == 0:
                segments[i].append(fv)
                assigned = True
                break
        if not assigned:
            segments[-1].append(fv)

    return segments


def optimal_k(feature_vectors):
    dbi_scores = []
    k_values = range(3, min(len(feature_vectors), 11))

    for k in k_values:
        gmm = GaussianMixture(n_components=k, random_state=0)
        gmm.fit(feature_vectors)

        dbi_score = davies_bouldin_score(
            feature_vectors, gmm.predict(feature_vectors))
        dbi_scores.append(dbi_score)

    dbi_scores = np.array(dbi_scores)
    index = np.argmin(dbi_scores)
    optimal_k = k_values[index]

    return optimal_k


def cluster(data):
    key_frames = []

    k = optimal_k(data)
    gmm = GaussianMixture(n_components=k)
    gmm.fit(data)

    centers = gmm.means_

    for i in range(len(centers)):
        key_array = centers[i]
        distances = cdist(data, np.expand_dims(
            key_array, axis=0), metric="euclidean")
        closest_index = np.argmin(distances)
        closest_array = data[closest_index]
        key_frames.append(closest_array)

    return key_frames


def generate_keyframes(img_features):
    sets = eratosthenis(img_features, 5)

    stage1 = []
    stage2 = []
    final_kf = []

    print("Performing Clustering Stage-I:\nSegments done: ", end="")
    for idx, set in enumerate(sets):
        print(idx + 1, " ", end="")
        var1 = cluster(set)

        listcopy(var1, stage1)

    print("\nPerforming Clustering Stage-II ")
    stage2 = cluster(stage1)

    for i in stage2:
        final_kf.append(frame_mapper(img_features, i))

    final_kf.sort()

    print(f"Keyframes are {final_kf}\n")

    return final_kf


def listcopy(l1, l2):
    for i in range(len(l1)):
        l2.append(l1[i])


def frame_mapper(dictionary, value):
    for key, val in dictionary.items():
        if np.array_equal(val, value):
            return key


def get_video_name_from_path(full_path):
    path, filename = os.path.split(full_path)
    name, extension = os.path.splitext(filename)

    return name


def calculate_similarity(feature_vector1, feature_vector2):
    similarity = cosine_similarity([feature_vector1], [feature_vector2])
    return similarity[0][0]

def boundary_determination(feature_vectors_dict, keyframes, initial_event_boundary_threshold):
    total_frames = len(feature_vectors_dict)
    max_total_length = int(total_frames * 0.15)

    print("\n")
    feature_vectors = list(feature_vectors_dict.values())
    merged_events = [(keyframe, keyframe) for keyframe in keyframes]
    total_length = len(keyframes)
    event_boundary_threshold = initial_event_boundary_threshold

    while total_length < max_total_length:
        next_events = [(start - 1, end + 1) for start, end in merged_events]
        next_total_length = sum(end_frame - start_frame + 1 for start_frame, end_frame in next_events)
        if next_total_length > max_total_length:
            event_boundary_threshold *= 0.9
        else:
            event_boundary_threshold *= 1.1

        merged_events = next_events
        total_length = next_total_length

    return merged_events,event_boundary_threshold



def generate_summary_selection(total_frames, frame_ranges):
    summary_selection = [0] * total_frames

    for start, end in frame_ranges:
        start = max(start, 0)
        end = min(end, total_frames)
        for i in range(start, end):
            summary_selection[i] = 1

    return summary_selection


def evaluate(summary_selection, videoName, HOMEDATA):
    gt_file = os.path.join(HOMEDATA, videoName + ".mat")
    gt_data = scipy.io.loadmat(gt_file)
    user_score = gt_data.get("user_score")
    nFrames = user_score.shape[0]
    nbOfUsers = user_score.shape[1]
    if len(summary_selection) < nFrames:
        warnings.warn(
            "Pad selection with %d zeros!" % (nFrames - len(summary_selection))
        )
        summary_selection.extend([0] * (nFrames - len(summary_selection)))
    elif len(summary_selection) > nFrames:
        warnings.warn(
            "Crop selection (%d frames) to GT length"
            % (len(summary_selection) - nFrames)
        )
        summary_selection = summary_selection[:nFrames]
    summary_indicator = np.array(
        [1 if x > 0 else 0 for x in summary_selection])
    user_intersection = np.zeros((nbOfUsers, 1))
    user_union = np.zeros((nbOfUsers, 1))
    user_length = np.zeros((nbOfUsers, 1))
    f_measure = []

    for userIdx in range(nbOfUsers):
        gt_indicator = np.array(
            [1 if x > 0 else 0 for x in user_score[:, userIdx]])

        user_intersection[userIdx] = np.sum(gt_indicator * summary_indicator)
        user_union[userIdx] = np.sum(
            np.array([1 if x > 0 else 0 for x in gt_indicator + summary_indicator])
        )
        user_length[userIdx] = np.sum(gt_indicator)

        if user_intersection[userIdx] > 0 or user_length[userIdx] > 0:
            f_measure.append(
                2
                * user_intersection[userIdx]
                / (user_length[userIdx] + user_intersection[userIdx])
            )
        else:
            f_measure.append(0)
    fscore = np.mean(f_measure)
    print("f-score=", fscore)
    return fscore


def save_details(file_path, video_details):
    is_empty = os.stat(file_path).st_size == 0

    with open(file_path, mode="a", newline="") as csvfile:
        writer = csv.writer(csvfile)
        if is_empty:
            header = [
                "Video Name",
                "Total Frames",
                "Keyframes",
                "Event Boundaries",
                "F-Score",
            ]
            writer.writerow(header)
        writer.writerow(video_details)


def output_summary(video_path, frame_ranges, output_path, video_name):
    video_clip = VideoFileClip(video_path)
    fps = video_clip.fps

    clips = []

    for start_frame, end_frame in frame_ranges:
        clip = mp.VideoFileClip(video_path).subclip(
            start_frame / fps, end_frame / fps)
        clips.append(clip)

    final_clip = mp.concatenate_videoclips(clips)
    # output_path1 = video_name + "_" + output_path
    final_clip.write_videofile(
        output_path, codec="libx264", audio_codec="aac")

In [ ]:
folder_path = r"/content/drive/MyDrive/Video_Summarization/videos"
video_files = glob.glob(
    os.path.join(folder_path, "*.mp4")
)
for x,i in enumerate(video_files):
  print(f"Video {x}:{ get_video_name_from_path(i)}")
Done=[0]


In [ ]:
HOMEDATA = "/content/drive/MyDrive/Video_Summarization/GT/"
event_boundary_threshold = 0.9985
for count, video_path in enumerate(video_files):
        videoName = get_video_name_from_path(video_path)

        print(f"Video {count+1}: {videoName}\n")
        img_features = extract_features_resnet18(video_path)

        keyframes = generate_keyframes(img_features)

        event_boundaries,ebt = boundary_determination(
            img_features, keyframes, event_boundary_threshold)

        total_frames = len(list(img_features.values()))
        summary_selection2 = generate_summary_selection(
            total_frames, event_boundaries)
        f_score = evaluate(summary_selection2, videoName, HOMEDATA)
        num=0
        for i in event_boundaries:
          i=list(i)
          down,up=i
          # print(up)
          num+=(up-down)
        video_details = [videoName, f_score,
                         total_frames, keyframes, event_boundaries,ebt,num]

        save_details("/content/drive/MyDrive/Video_Summarization/video_summary_data.csv", video_details)
        # output_path="/content/drive/MyDrive/Video_Summarization/Summaries/"+videoName+"_summary.mp4"
        # output_summary(video_path, event_boundaries,output_path, videoName)
        print("\n----------------------------------------------------------------------------------------------------------------------\n")



# print(f_score)

Video 3: Bearpark_climbing

Frames per second:  25.0


Extracting Features:   1%|          | 28/3341 [00:04<08:28,  6.52it/s]


KeyboardInterrupt: ignored

In [ ]:
count=0
video_path=video_files[0]
    # if count%2==0 and count not in Done:
videoName = get_video_name_from_path(video_path)
print(f"Video {count+1}: {videoName}\n")
# img_features = extract_features_resnet18(video_path)
keyframes = generate_keyframes(img_features)
event_boundaries,ebt = boundary_determination(
    img_features, keyframes, event_boundary_threshold)
total_frames = len(list(img_features.values()))
summary_selection2 = generate_summary_selection(
    total_frames, event_boundaries)
f_score = evaluate(summary_selection2, videoName, HOMEDATA)
num=0
for i in event_boundaries:
  i=list(i)
  down,up=i
  # print(up)
  num+=(up-down)
video_details = [videoName, f_score,
                 total_frames, keyframes, event_boundaries,ebt,num]
save_details("/content/drive/MyDrive/Video_Summarization/video_summary_data.csv", video_details)

In [ ]:
HOMEDATA = "/content/drive/MyDrive/Video_Summarization/GT/"
event_boundary_threshold = 0.9985
for count, video_path in enumerate(video_files):
    if count%2!=0 and count not in Done:
        videoName = get_video_name_from_path(video_path)

        print(f"Video {count+1}: {videoName}\n")
        img_features = extract_features_resnet18(video_path)

        keyframes = generate_keyframes(img_features)

        event_boundaries,ebt = boundary_determination(
            img_features, keyframes, event_boundary_threshold)

        total_frames = len(list(img_features.values()))
        summary_selection2 = generate_summary_selection(
            total_frames, event_boundaries)
        f_score = evaluate(summary_selection2, videoName, HOMEDATA)
        num=0
        for i in event_boundaries:
          i=list(i)
          down,up=i
          # print(up)
          num+=(up-down)
        video_details = [videoName, f_score,
                         total_frames, keyframes, event_boundaries,ebt,num]

        save_details("/content/drive/MyDrive/Video_Summarization/video_summary_data.csv", video_details)
        # output_path="/content/drive/MyDrive/Video_Summarization/Summaries/"+videoName+"_summary.mp4"
        # output_summary(video_path, event_boundaries,output_path, videoName)
        print("\n----------------------------------------------------------------------------------------------------------------------\n")



# print(f_score)

In [ ]:

for count, video_path in enumerate(video_files):
    if count==13 and count not in Done:
        videoName = get_video_name_from_path(video_path)

        print(f"Video {count+1}: {videoName}\n")
        img_features = extract_features_resnet18(video_path)

        keyframes = generate_keyframes(img_features)




# print(f_score)

In [ ]:
HOMEDATA = "/content/drive/MyDrive/Video_Summarization/GT/"
event_boundary_threshold = 0.9985
event_boundaries,ebt = boundary_determination(img_features, keyframes, event_boundary_threshold)
total_frames = len(list(img_features.values()))

summary_selection2 = generate_summary_selection(total_frames, event_boundaries)
f_score = evaluate(summary_selection2, videoName, HOMEDATA)

num=0

for i in event_boundaries:

        i=list(i)

        down,up=i

        # print(up)

        num+=(up-down)

video_details = [videoName, f_score, total_frames, keyframes, event_boundaries,ebt,num]
# save_details("/content/drive/MyDrive/Video_Summarization/video_summary_data.csv", video_details)
output_path="/content/drive/MyDrive/Video_Summarization/Summaries/"+videoName+"_summary.mp4"
output_summary(video_path, event_boundaries,output_path, videoName)
print("\n----------------------------------------------------------------------------------------------------------------------\n")


In [ ]:
HOMEDATA = "/content/drive/MyDrive/Video_Summarization/GT/"
event_boundary_threshold = 0.9985
for count, video_path in enumerate(video_files):
    if count==7 and count not in Done:
        videoName = get_video_name_from_path(video_path)

        print(f"Video {count+1}: {videoName}\n")
        # img_features = extract_features_resnet18(video_path)

        keyframes = [125, 328, 413, 427, 539, 566, 703, 712, 775, 959]

        # event_boundaries,ebt = boundary_determination(
        #     img_features, keyframes, event_boundary_threshold)
        event_boundaries=[(115, 135), (318, 338), (403, 423), (417, 437), (529, 549), (556, 576), (693, 713), (702, 722), (765, 785), (949, 969)]
        # total_frames = len(list(img_features.values()))
        summary_selection2 = generate_summary_selection(
            total_frames, event_boundaries)
        f_score = evaluate(summary_selection2, videoName, HOMEDATA)
        num=0
        for i in event_boundaries:
          i=list(i)
          down,up=i
          # print(up)
          num+=(up-down)
        video_details = [videoName, f_score,
                         total_frames, keyframes, event_boundaries,ebt,num]

        save_details("/content/drive/MyDrive/Video_Summarization/video_summary_data.csv", video_details)
        output_path="/content/drive/MyDrive/Video_Summarization/Summaries/"+videoName+"_summary.mp4"
        output_summary(video_path, event_boundaries,output_path, videoName)
        print("\n----------------------------------------------------------------------------------------------------------------------\n")


